C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi agenzie e reseller\Analisi agenzie

In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from os import listdir
from os.path import isfile, join
from os.path import getmtime

from zebra import remov_duplicates, human_format
from babel.numbers import format_number

In [2]:
idx = pd.IndexSlice

In [3]:
pd.set_option('display.max_rows', 100)

# Sources

## Parametriche

In [4]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)

In [5]:
cod_prod = par["Classificazione_codice_prodotto"]
cod_prod["Codice PRODOTTO"] = cod_prod["Codice PRODOTTO"].apply(lambda x: str(x))
cod_prod = cod_prod.loc[:, ['MDM', 'GTM BDG', 'Solution','Sub solution',"Codice PRODOTTO"]]

sales_director = par["Sales_director"]
sales_director = sales_director.loc[:, ["Sales Director", "Area territoriale", "Rete"]]
sales_director["Sales Director"] = sales_director["Sales Director"].str.lower()

agente_ordine = par["Agente_ordine"]

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

li_prodotti = par["Linea_prodotti"]

## Parco clienti da Licenze e abbonamenti

In [35]:
clienti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Abbonamenti\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Abbonamenti") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Abbonamenti", f))]

dfs = []
for file in onlyfiles:
    data = pd.read_excel(clienti+file, na_values="#", dtype={"Data inizio contratt":object, "Data Inizio Abb.":object, "Data Fine Abb.":object, "Cliente FATTURA":str, "Codice ABBONAMENTO":str})
    data["Tipo Licenza"] = file.split("_")[1].split(".")[0]
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(clienti+file))
    dfs.append(data)

cli = pd.concat(dfs, axis=0, ignore_index=True)
    
for col in cli.columns:
    if "Data" in col:
        
        cli[col] = cli[col].apply(lambda x: (float(x) - 25569) * 86400)
        cli[col] = pd.to_datetime(cli[col], unit="s")
    else:
        pass

cli.rename(columns={'Unnamed: 10':'Descrizione ABBONAMENTO'}, inplace=True )

## Raccolto da Budest

In [6]:
raccolto = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Budest\Budest cliente prodotto anno\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Budest\Budest cliente prodotto anno") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Budest\Budest cliente prodotto anno", f))]

dfs = []
for file in onlyfiles:
    data = pd.read_excel(raccolto+file, na_values="#", dtype={"Cliente MERCE":str, "Codice ABBONAMENTO":str})
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(raccolto+file))
    dfs.append(data)

fatt = pd.concat(dfs, axis=0, ignore_index=True)

In [7]:
fatt.rename(columns={'Unnamed: 3':'Descrizione Agente', 'Unnamed: 7':'Descrizione PRODOTTO'}, inplace=True )

## All accounts da Salesforce

In [ ]:
accounts_sf = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_all_accounts.csv", encoding='latin-1', sep=";")

# Analisi

## Raccolto

In [52]:
sales_dir = ["ARTEL FORFAIT","ARTEL INDIRETTA","IPSOA RETE","OSRA FORFAIT","OSRA INDIRETTA","RETE WKI","Senza attribuz.","PRAGMA"] 
columns = ["Cliente Merce", "Segmento Aggregato","Descrizione Agente", "RSM Agente", "Area territoriale", "Rete", "Area Prodotto", "Linea Prodotti", "Fornitura (SW)", "Solution", "GTM BDG",  "Anno Inserimento Ord", "Raccolto"]

def merge_with_product_and_salesdirector(df):

    df = fatt.loc[fatt["Sales Director"].isin(sales_dir) ,:].dropna(axis=0, subset=["Raccolto"]).copy()
    df["Cliente Merce"] = df["Cliente Merce"].astype(int).astype(str)
    df["Codice PRODOTTO"] = df["Codice PRODOTTO"].astype(str)
    df["Sales Director"] = df["Sales Director"].apply(lambda x: x.lower())
    

    df = df.merge(cod_prod, how="left", on="Codice PRODOTTO").merge(sales_director, how="left", on="Sales Director")
  
    return df.loc[:, columns]

fatt_ind = fatt.pipe(merge_with_product_and_salesdirector)


In [53]:
fatt_ind["Area Prodotto"] = np.select(
    [fatt_ind["Area Prodotto"] == "Rinnovi Software", 
    fatt_ind["Area Prodotto"] == "Altro",
    fatt_ind["Area Prodotto"] == "Vendite Software", 
    fatt_ind["Area Prodotto"] == "Vendite Editoria"],
    ["Rinnovi", "Rinnovi", "Nuove vendite", "Nuove vendite"]
    )

In [61]:
fatt_ind.rename(columns={"Descrizione Agente": "Agente", "Anno Inserimento Ord": "Anno"}, inplace=True)

In [62]:
fatt_ind.head()

,Cliente Merce,Segmento Aggregato,Agente,RSM Agente,Area territoriale,Rete,Area Prodotto,Linea Prodotti,Fornitura (SW),Solution,GTM BDG,Anno,Raccolto
0,40,Studi Fiscali,VERSYA S.P.A.,VINCENZI CRISTINA,Rete IPSOA,Indiretta agenzie,Rinnovi,Software,ON PREMISE,Sistema Professionista,Legacy Tax,2018,3025.97
1,40,Studi Fiscali,VERSYA S.P.A.,VINCENZI CRISTINA,Rete IPSOA,Indiretta agenzie,Rinnovi,Software,ON PREMISE,Sistema Professionista,Legacy Tax,2018,658.00
2,40,Studi Fiscali,VERSYA S.P.A.,VINCENZI CRISTINA,Rete IPSOA,Indiretta agenzie,Rinnovi,Software,ON PREMISE,Sistema Professionista,Legacy Tax,2018,628.12
3,40,Studi Fiscali,VERSYA S.P.A.,VINCENZI CRISTINA,Rete IPSOA,Indiretta agenzie,Rinnovi,Software,ON PREMISE,Siar,Legacy Tax,2018,448.66
4,40,Studi Fiscali,VERSYA S.P.A.,VINCENZI CRISTINA,Rete IPSOA,Indiretta agenzie,Rinnovi,Software,ON PREMISE,Dichiarazioni Stand Alone,Legacy Tax,2018,169.55


## Abbonamenti

In [58]:
stato_abb = (cli["Stato Esteso"].str.contains("ATTIVO")) & (cli["Stato Sintetico Contratto"].isin(["Attivo","Sospeso"]))
columns = ["Cliente MERCE","Segmento Aggregato Cl.Merce",'Regione', 'Provincia', "Agente dell'ORDINE", "RSM Agente", "Rete", "Area territoriale",'Solution', "GTM BDG","Fornitura (SW)","Valore PV New"]

In [59]:
def creazione_clienti(df):
    
    df = cli.loc[stato_abb & (cli["Sales Director"].isin(sales_dir)) ,:].dropna(axis=0, subset=["Valore PV New"]).copy()
    
    df["Sales Director"] = df["Sales Director"].str.lower()
    df["Codice ABBONAMENTO"] = df.loc[:,"Codice ABBONAMENTO"].apply(str)

    df = df.merge(cod_prod, how="left", left_on="Codice ABBONAMENTO", right_on="Codice PRODOTTO")\
          .merge(sales_director, how="left", on="Sales Director")\
          .merge(agente_ordine, how="left", left_on="Agente dell'Abbonamento", right_on="Codice agente")
    df["Cliente MERCE"] = df["Cliente MERCE"].apply(str)
    
    return df.loc[:, columns]

clienti = cli.pipe(creazione_clienti)
clienti.rename(columns={"Cliente MERCE": "Cliente Merce", "Valore PV New": "Valore PV", "Segmento Aggregato Cl.Merce":"Segmento Aggregato", "Agente dell'ORDINE":"Agente"}, inplace=True)

In [112]:
anagrafiche = fatt_ind.groupby(["Cliente Merce"], dropna=False).size().reset_index()\
    .merge(
        clienti.groupby(["Cliente Merce", "Provincia", "Regione"], dropna=False).size().reset_index(), 
        on= ["Cliente Merce"], 
        how="outer", indicator=True)\
            .sort_values(by=["Cliente Merce"], ascending=False)

## Output

In [ ]:
anagrafiche.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi agenzie e reseller\Analisi agenzie\Data source\anagrafiche.xlsx", index=False)
clienti.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi agenzie e reseller\Analisi agenzie\Data source\clienti.xlsx", index=False)
fatt_ind.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi agenzie e reseller\Analisi agenzie\Data source\fatturato_finanziario.xlsx", index=False)